In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 

In [2]:
import os
from getpass import getpass


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")


Enter your Mistral API Key: ··········


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Day 14 Query-Transformation-in-RAG.pdf")  # upload PDF to Colab
documents = loader.load()

print(f"Total pages loaded: {len(documents)}")


Total pages loaded: 10


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 16


In [10]:
from langchain_mistralai.embeddings import MistralAIEmbeddings

embedding_model = MistralAIEmbeddings(
    model="mistral-embed",
    api_key="RCXQw2QTzxrGglVQde2oMtTKudKRYXo0"
)


In [11]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print("Vector store created successfully")


Vector store created successfully


In [12]:
query = "What is this document about?"

results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:400])



--- Chunk 1 ---
links between events, leading to 
comprehensive understanding.
Relationships Entities or concepts are rarely described 
in the same document chunk.
Allows separate retrieval for each entity, 
then linking them contextually.
Comparisons Documents seldom contain direct 
comparative analyses of multiple 
subjects.
Facilitates two or more separate retrievals 
for each item, then comparison.

--- Chunk 2 ---
initiatives”
 More precise
  Better matching to document language
  Higher retrieval accuracy

--- Chunk 3 ---
Sub-Query 2
What is the current status of NASA's lunar missions?
Sub-Query 3
How did those specific collaborations influence the progress of NASA's lunar 
missions?
The LLM retrieves context for each sub-query, then intelligently merges these findings to form a 
comprehensive, coherent final answer.


In [34]:
from langchain_mistralai.chat_models import ChatMistralAI


llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    api_key="RCXQw2QTzxrGglVQde2oMtTKudKRYXo0"
)


In [24]:
def generate_answer(query, retrieved_docs):
    context = "\n\n".join(
        [f"[Chunk {i+1}]: {doc.page_content}"
         for i, doc in enumerate(retrieved_docs)]
    )

    prompt = f"""
You are a factual assistant.

Answer the question using ONLY the context below.
If the answer is not present, say:
"Answer not found in the provided document."

CONTEXT:
{context}

QUESTION:
{query}

Include citations like [Chunk X].
"""

    response = llm.invoke(prompt)
    return response.content


In [50]:
SIMILARITY_THRESHOLD = 0.5

def retrieve_with_threshold(query, k=5):
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k)

    filtered_docs = []
    for doc, score in docs_with_scores:
        similarity = 1 / (1 + score)  # FAISS distance → similarity
        if similarity >= SIMILARITY_THRESHOLD:
            filtered_docs.append(doc)

    return filtered_docs


In [36]:
def rag_pipeline(query):
    retrieved_docs = retrieve_with_threshold(query)

    if len(retrieved_docs) == 0:
        return "Naku telidu mastaru !"

    return generate_answer(query, retrieved_docs)


In [51]:
query = "How Multi-Hop Retrieval Works?"
answer = rag_pipeline(query)
print(answer)


Here’s how **Multi-Hop Retrieval** works, based on the provided context:

1. **Receive Question**: The initial complex user query is processed [Chunk 1].
2. **Decompose Queries**: The main question is broken down into smaller, actionable sub-queries [Chunk 1, Chunk 3].
3. **Retrieve Context**: Relevant information is fetched for each individual sub-query [Chunk 1].
4. **Aggregate Evidence**: All retrieved contexts are synthesized and combined [Chunk 1].
5. **Generate Answer**: A final, well-reasoned answer is formulated from the aggregated evidence [Chunk 1].

**Key Benefits**:
- Enhanced recall and higher factual accuracy [Chunk 1].

Citations: [Chunk 1], [Chunk 3].
